In [4]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      #'Azure region: ' + ws.location, 
      #'Subscription id: ' + ws.subscription_id, 
      #'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code DNX59EUUA to authenticate.


AuthenticationException: AuthenticationException:
	Message: Unknown error occurred during authentication. Error detail: Unexpected polling state interaction_required
	InnerException None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "inner_error": {
            "code": "Authentication"
        },
        "message": "Unknown error occurred during authentication. Error detail: Unexpected polling state interaction_required"
    }
}

In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException 

# Choose a name for your CPU cluster
amlcompute_cluster_name = "Demo-Cluster"

# Verify that cluster does not exist already
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print("Creating a new compute cluster...")
    compute_config =  AmlCompute.provisioning_configuration(vm_size='STANDARD_D2S_V3', max_nodes=4, min_nodes=0, vm_priority='lowpriority') 
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
    
#aml_compute.wait_for_completion(show_output=True)
aml_compute.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)
#compute_target.wait_for_completion(show_output=True, min_node_count = None, timeout_in_minutes = 60)


# use get_status() to get a detailed status for the current cluster. 
print(aml_compute.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-10-06T07:03:13.637000+00:00', 'errors': None, 'creationTime': '2021-09-17T05:18:08.686648+00:00', 'modifiedTime': '2021-09-17T05:18:24.335748+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 1, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2S_V3'}


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform , choice
from azureml.train.estimator import Estimator
import os

# Specify parameter sampler
paraSampler = RandomParameterSampling(
             {
            "--C": uniform(0.1, 1.0),
              } )


# Specify a Policy
policyBandit = BanditPolicy(slack_factor = 0.1, evaluation_interval=1, delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")


    
# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=aml_compute, entry_script="train.py",
              script_params={'--C':1.0, '--max_iter':100})    

# Create a SKLearn estimator for use with train.py
#est =  Estimator(source_directory=os.path.join('.'),
#                      entry_script='train.py',
#                      compute_target=aml_compute)


# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                    estimator=est,
                    hyperparameter_sampling=paraSampler,
                    policy=policyBandit,
                    primary_metric_name='Accuracy',
                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                    max_concurrent_runs = 4,    
                    max_total_runs=40)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

#exp = Experiment(workspace=ws, name='tuning')
#run = exp.submit(hyperdrive_config)

hyperdrive_run = exp.submit(config=hyperdrive_config, show_output=True)
RunDetails(hyperdrive_run).show()

#run.wait_for_completion(show_output=True)

### YOUR CODE HERE ###

The same input parameter(s) are specified in estimator/run_config script params and HyperDrive parameter space. HyperDrive parameter space definition will override these duplicate entries. ['--C'] is the list of overridden parameter(s).


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [9]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

In [10]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
print(best_run.get_details()['runDefinition']['arguments'])
print(best_run.get_file_names())
model = best_run.register_model(model_name = "Best", model_path=".")

best_run_metrics = best_run.get_metrics()
print('\n Accuracy: ', best_run_metrics['Accuracy'] )

### YOUR CODE HERE ###

['--max_iter', '100', '--C', '0.10145586951662584']
['azureml-logs/55_azureml-execution-tvmps_c68a38cf75a0b43546ee210f6695b8b47c6de3eb607fd2136919179b646c529a_d.txt', 'azureml-logs/65_job_prep-tvmps_c68a38cf75a0b43546ee210f6695b8b47c6de3eb607fd2136919179b646c529a_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_c68a38cf75a0b43546ee210f6695b8b47c6de3eb607fd2136919179b646c529a_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log', 'outputs/model.joblib']

 Accuracy:  0.9102446201949604


## AUTO ML 

In [11]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(url)

### YOUR CODE HERE ###

In [26]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd
from azureml.core import Dataset

x, y = clean_data(ds)

y_dataframe = pd.DataFrame(y,columns=['y'])
all_data = pd.concat([x,y_dataframe], axis = 1)

x_train, x_test = train_test_split(all_data, test_size=0.2, random_state=1)

x_train.to_csv("training/train_data.csv", index = False)

data_store = ws.get_default_datastore()
data_store.upload(src_dir = "./training", target_path = 'udacity-project', overwrite = True,  show_progress = True)
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

Uploading an estimated of 5 files
Uploading ./training/.amlignore
Uploaded ./training/.amlignore, 1 files out of an estimated total of 5
Uploading ./training/.amlignore.amltmp
Uploaded ./training/.amlignore.amltmp, 2 files out of an estimated total of 5
Uploading ./training/train.py
Uploaded ./training/train.py, 3 files out of an estimated total of 5
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 4 files out of an estimated total of 5
Uploading ./training/.ipynb_checkpoints/train-checkpoint.py
Uploaded ./training/.ipynb_checkpoints/train-checkpoint.py, 5 files out of an estimated total of 5
Uploaded 5 files


In [28]:
y_dataframe.head(2)

,y
0,0
1,0


In [29]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=3,
    compute_target=aml_compute)

In [31]:
# Submit your automl run
automl_run = exp.submit(automl_config,show_output = True)
RunDetails(automl_run).show()
automl_run.wait_for_completion()


Submitting remote run.
Running on remote compute: Demo-Cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_ffb394ea-c20b-422b-8076-8848bcef6791,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

{'runId': 'AutoML_ffb394ea-c20b-422b-8076-8848bcef6791',
 'target': 'Demo-Cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-10-06T09:54:00.91098Z',
 'endTimeUtc': '2021-10-06T10:34:20.622076Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '3',
  'target': 'Demo-Cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-project","subscription_id":"73f88e6b-3a35-4612-b550-555157e7059f","resource_group":"ops_analytics-rg-gb-dev","workspace_name":"ops_analytics-workspace","region":"westeurope","compute_target":"Demo-Cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"enable_code_generation":false,"iterations":1000,"primary_metric":"accuracy",

In [34]:
# Retrieve and save your best automl model.
### YOUR CODE HERE ###

best_model, fitted_model = automl_run.get_output()
#print(best_model)
print(fitted_model)

Package:azureml-automl-runtime, training version:1.34.1, current version:1.33.0
Package:azureml-core, training version:1.34.0, current version:1.33.0
Package:azureml-dataprep, training version:2.22.3, current version:2.20.1
Package:azureml-dataprep-rslex, training version:1.20.3, current version:1.18.0
Package:azureml-dataset-runtime, training version:1.34.0, current version:1.33.0
Package:azureml-defaults, training version:1.34.0, current version:1.33.0
Package:azureml-interpret, training version:1.34.0, current version:1.33.0
Package:azureml-mlflow, training version:1.34.0, current version:1.33.0
Package:azureml-pipeline-core, training version:1.34.0, current version:1.33.0
Package:azureml-responsibleai, training version:1.34.0, current version:1.33.0
Package:azureml-telemetry, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-client, training version:1.34.0, current version:1.33.0
Package:azureml-train-automl-runtime, training version:1.34.1, current versi

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.7, tree_method='auto'))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855, learning_rate='constant', loss='modified_huber', max_iter=1000, n_jobs=1, penalty='none', power_t=0.7777777777777777, random_state=None, tol=0.0001))], verbose=False))], flatten_transform=None, weights=[0.23076923076923078, 0.07692307692307693, 0.0769

In [35]:
fitted_model.named_steps['datatransformer']. get_engineered_feature_names ()

['age_MeanImputer',
 'duration_MeanImputer',
 'emp.var.rate_MeanImputer',
 'cons.price.idx_MeanImputer',
 'cons.conf.idx_MeanImputer',
 'euribor3m_MeanImputer',
 'nr.employed_MeanImputer',
 'marital_ModeCatImputer_LabelEncoder',
 'default_ModeCatImputer_LabelEncoder',
 'housing_ModeCatImputer_LabelEncoder',
 'loan_ModeCatImputer_LabelEncoder',
 'month_CharGramCountVectorizer_10',
 'month_CharGramCountVectorizer_11',
 'month_CharGramCountVectorizer_12',
 'month_CharGramCountVectorizer_3',
 'month_CharGramCountVectorizer_4',
 'month_CharGramCountVectorizer_5',
 'month_CharGramCountVectorizer_6',
 'month_CharGramCountVectorizer_7',
 'month_CharGramCountVectorizer_8',
 'month_CharGramCountVectorizer_9',
 'day_of_week_CharGramCountVectorizer_1',
 'day_of_week_CharGramCountVectorizer_2',
 'day_of_week_CharGramCountVectorizer_3',
 'day_of_week_CharGramCountVectorizer_4',
 'day_of_week_CharGramCountVectorizer_5',
 'campaign_CharGramCountVectorizer_1',
 'campaign_CharGramCountVectorizer_10',
 '

In [36]:
fitted_model.named_steps['datatransformer'].get_featurization_summary()

[{'RawFeatureName': 'age',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'duration',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'emp.var.rate',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'cons.price.idx',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'cons.conf.idx',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'euribor3m',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'EngineeredFeatureCount': 1,
  'Transformations': ['MeanImputer']},
 {'RawFeatureName': 'nr.employed',
  'TypeDetected': 'Numeric',
  'Dropped': 'No',
  'Eng